In [1]:
import time
import random
import datetime
import tqdm
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import pyro
import pyro.contrib.examples.polyphonic_data_loader as poly
import pyro.distributions as dist
import pyro.poutine as poutine
from pyro.distributions import TransformedDistribution
from pyro.distributions.transforms import affine_autoregressive

import ot
import torch.nn.functional as F

# DATA

In [2]:
def easyTones():
    training_seq_lengths = torch.tensor([8]*10)
    training_data_sequences = torch.zeros(10,8,88)
    for i in range(8):
        training_data_sequences[i][0][int(70-i*10)  ] = 1
        training_data_sequences[i][1][int(70-i*10)+2] = 1
        training_data_sequences[i][2][int(70-i*10)+4] = 1
        training_data_sequences[i][3][int(70-i*10)+5] = 1
        training_data_sequences[i][4][int(70-i*10)+7] = 1
        training_data_sequences[i][5][int(70-i*10)+9] = 1
        training_data_sequences[i][6][int(70-i*10)+11] = 1
        training_data_sequences[i][7][int(70-i*10)+12] = 1
    return training_seq_lengths, training_data_sequences
## ドドド、レレレ
def superEasyTones():
    training_seq_lengths = torch.tensor([8]*10)
    training_data_sequences = torch.zeros(10,8,88)
    for i in range(10):
        for j in range(8):
            training_data_sequences[i][j][int(30+i*5)] = 1
    return training_seq_lengths, training_data_sequences
## ドドド、ドドド、ドドド
def easiestTones():
    training_seq_lengths = torch.tensor([8]*10)
    training_data_sequences = torch.zeros(10,8,88)
    for i in range(10):
        for j in range(8):
            training_data_sequences[i][j][int(70)] = 1
    return training_seq_lengths, training_data_sequences

# WASSERSTEIN

In [3]:
def EMD2(xT, mud_arr):
    # print("DONE0")
    x_p = xT.detach().clone().numpy()
    mud_p = mud_arr.detach().clone().numpy()
    
    n_points = xT.size()[0]
    n_tones = xT.size()[1]
    dim = xT.size()[2]
    # print("DONE1")
    # x_p = xT.reshape((n_points, dim)).cpu().detach().numpy()
    x_p = x_p.reshape((n_points, n_tones * dim))
    # mud_p = mud_arr.reshape((n_points, dim)).cpu().detach().numpy()
    mud_p = mud_p.reshape((n_points, n_tones * dim))
    # print(x_p.shape)
    # print(mud_p.shape)
    # print("DONE2")
    M = ot.dist(x_p, mud_p)
    M /= M.max()
    a, b = np.ones((n_points,)) / n_points, np.ones((n_points,)) / n_points  # uniform distribution on samples
    gamma = ot.emd(a, b, M)
    # wdis = ot.emd2_1d(xT, mud_arr)
    # print('Wasserstein distance:', wdis)
    # print("DONE3")

    Tx = []
    for i, xTi in enumerate(x_p):
        ind = gamma[i,:].argmax()
        Tx.append(mud_arr[ind])
        # print(f'index {i}: ind {ind} mud_p[{ind}]={mud_p[ind]}') 
    # print("DONE4")
    # print(Tx[0])
    # Tx = torch.from_numpy(np.array(Tx).reshape((n_points, 1, n_tones*dim)))
    # Tx = torch.tensor(Tx,dtype=torch.float32)
    # x = xT.reshape(n_points,1,n_tones*dim)
    # Wasser = F.mse_loss(x, Tx)
    Tx = torch.stack(Tx)
    return torch.norm(Tx-xT, dim=1).sum()

In [96]:
_, song_1 = easiestTones()
_, song_2 = superEasyTones()
_, song_2 = easiestTones()
song_2.requires_grad=True
# print(song_2[0][1][30])
song_2[0][1][30] = 1.
now = time.time()
print(EMD2(song_1, song_2))
print(time.time() - now)

tensor(1., grad_fn=<SumBackward0>)
0.003001689910888672


In [5]:
a = torch.tensor([[3., 4.], [3., 4.]])
b = torch.tensor([1])
wass = torch.norm(a, dim=1)
print(wass.sum())

tensor(10.)
